Para o relatório parcial na foi necessário ser feita a reprodução do método de identificação proposto no trabalho da Carolina para o modelo MP.

## Modelo MP:

\begin{equation}
\tilde{y}(n) = \sum^P_{p=1}\sum^M_{m=0} \tilde{b}_{mp} \tilde{x}(n-m) |\tilde{x}(n-m)|^{p-1} 
\end{equation}


In [20]:
# IMPORTAÇÃO DAS BIBLIOTECAS E DADOS

import scipy.io as spio
import scipy as sp
import numpy as np
import matplotlib.pyplot as plt


mat = spio.loadmat('data_LDMOS.mat')
in_ext = mat['in_extraction']
out_ext = mat['out_extraction']
in_val = mat['in_validation']
out_val = mat['out_validation']

\begin{equation}
f^{pol}_{m}(|\tilde{x}(n-m)|)~f^{LUT}_{m}(|\tilde{x}(n-m)|) = \tilde{s}_{m(q-1)} + \left[ \frac{\tilde{s}_{m(q)} - \tilde{s}_{m(q-1)}}{e_{m(q)} - e_{m(q-1)}} \right] [|\tilde{x}(n-m)| - e_{m(q-1)}]
\end{equation}

In [85]:
# DEFINIÇÃO DOS PARÂMETROS DO MODELO
P = 3
M = 1
n = 2

# DEFINIÇÃO DOS VALORES DE ENTRADA E SAÍDA DAS LUTS
def saida_lut(entrada,M,n):
    entrada_m = zeros((len(entrada),M+1))
    Q = 2**n
    z=np.linspace(0,1,Q)
    z = [int(round(a*(len(entrada)-1))) for a in z]  
    s_lut = np.zeros((Q))
    

    s_lut = np.zeros((Q,2))

    for r in range(Q):
        s_lut[r,0] = np.real(entrada[z[r]])
        s_lut[r,1] = np.imag(entrada[z[r]])

    return s_lut
s_lut = saida_lut(in_ext,M,n)
s_lut

array([[ 0.06874848, -0.02982907],
       [-0.04522785,  0.22038117],
       [-0.08282102, -0.18595306],
       [ 0.0849741 , -0.34643166]])